In [ ]:
import numpy as np
import cupy as cp
import sys
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
from types import SimpleNamespace
import h5py
import dxchange
import warnings
warnings.filterwarnings("ignore", message=f".*peer.*")

sys.path.insert(0, '..')
from utils import *
from rec import Rec

In [ ]:
bin = 0
ntheta = 2400
path = f'/data/vnikitin/ESRF/ID16A/20240924_rec_ca/data/'
with  h5py.File(f'{path}/data_atomium.h5','r') as fid:
    data = fid[f'/exchange/pdata'][::7200//ntheta].astype('float32')
    psi_init = fid[f'/exchange/psi_init'][::7200//ntheta].astype('complex64')
    ref0 = fid[f'/exchange/pref0'][:].astype('float32')
    ref1 = fid[f'/exchange/pref1'][:].astype('float32')
    theta = fid[f'/exchange/theta'][::7200//ntheta].astype('float32')
    code = fid[f'/exchange/code'][:].astype('complex64')
    q = fid[f'/exchange/prb'][:].astype('complex64')
    
    shifts_cor = fid[f'/exchange/shifts_cor'][::7200//ntheta].astype('float32')
    shifts_code = fid[f'/exchange/shifts_code'][::7200//ntheta].astype('float32')
    cdata = fid['/exchange/cdata'][::7200//ntheta]
    
    z1 = fid['/exchange/z1'][0]
    detector_pixelsize = fid['/exchange/detector_pixelsize'][0]
    focusdetectordistance = fid['/exchange/focusdetectordistance'][0]
    energy = fid['/exchange/energy'][0]


with  h5py.File(f'{path}/data_ca.h5','r') as fid:
    z1c = fid['/exchange/z1'][0] 

for k in range(bin):
    data = 0.5*(data[:,::2]+data[:,1::2])
    data = 0.5*(data[:,:,::2]+data[:,:,1::2])
    psi_init = 0.5*(psi_init[:,::2]+psi_init[:,1::2])
    psi_init = 0.5*(psi_init[:,:,::2]+psi_init[:,:,1::2])
    ref0 = 0.5*(ref0[::2]+ref0[1::2])
    ref0 = 0.5*(ref0[:,::2]+ref0[:,1::2])
    cdata = 0.5*(cdata[::2]+cdata[1::2])
    cdata = 0.5*(cdata[:,::2]+cdata[:,1::2])
    q = 0.5*(q[::2]+q[1::2])
    q = 0.5*(q[:,::2]+q[:,1::2])
    code = 0.5*(code[::2]+code[1::2])
    code = 0.5*(code[:,::2]+code[:,1::2])
shifts_cor/=2**bin
shifts_code/=2**bin

In [ ]:
wavelength = 1.24e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
z2 = focusToDetectorDistance-z1
distance = (z1*z2)/focusToDetectorDistance
magnification = focusToDetectorDistance/z1
voxelsize = np.abs(detector_pixelsize/magnification)  # object voxel size
magnifications2 = z1/z1c
distancec = (z1-z1c)/(z1c/z1)
show = True

In [ ]:
args = SimpleNamespace()
args.ngpus = 4
args.lam = 50

args.n = 2048//2**bin
voxelsize *= 2048/args.n

args.ntheta = ntheta
args.ncode = 8192*args.n//2048
args.pad = 0
args.npsi = args.n + 2 * args.pad
args.nq = args.n + 2 * args.pad
args.ex = 32
args.npatch = args.nq + 2 * args.ex
args.nchunk = 16

args.voxelsize = voxelsize
args.wavelength = wavelength
args.distance = distance
args.distancec = distancec
args.eps = 1e-8
args.rho = [1, 20, 10]
args.path_out = f"/data/vnikitin/ESRF/ID16A/20240924_rec_ca/rec_atomium/r_{args.lam}_{args.pad}"
args.show = show

args.niter=4097
args.vis_step=4
args.err_step=4
args.rotation_axis=756*args.n/1024-args.n//4+args.pad

args.theta = theta*np.pi/180
# create class
cl_rec = Rec(args)


In [ ]:
psi_data = cl_rec.logR(psi_init)

In [ ]:
u_init = cl_rec.rec_tomo(psi_data,shifts_cor,32)
mshow_complex(u_init[u_init.shape[0]//2],args.show)

In [ ]:
path = f'/data/vnikitin/ESRF/ID16A/20240924_rec_ca/data/'
with  h5py.File(f'{path}/data_atomium.h5','a') as fid:
    try:
        del fid['/exchange/u_init']        
    except:
        pass
    fid.create_dataset('/exchange/u_init',data=cdata)
    